<a href="https://colab.research.google.com/github/Pratiksha100/AnalyticsProject/blob/akshay%2F443308/Concord_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/jvkersch/pyconcorde

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/jvkersch/pyconcorde to /tmp/pip-req-build-ufdfwkrz
  Running command git clone --filter=blob:none --quiet https://github.com/jvkersch/pyconcorde /tmp/pip-req-build-ufdfwkrz
  Resolved https://github.com/jvkersch/pyconcorde to commit 71232a8043a3ca79a7a6da2f48e9d89d81d37163
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from google.colab import drive
import numpy as np
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#Testing the concord solver on a sample file
from concorde.tsp import TSPSolver
from concorde.tests.data_utils import get_dataset_path

fname = '/content/drive/MyDrive/analytics/si1000.tsp'
solver = TSPSolver.from_tspfile(fname)
solution = solver.solve()

print(solution.found_tour)

print(solution.optimal_value)

print(solution.tour)

True
92650.0
[0 1 2 ... 5 4 3]


In [4]:
def create_dist_mat_from_atsp(path_of_file):
  '''
  input : path of the .atsp file
  output : returns  a distance matrix 
  ''' 

  # specify the path to the .atsp file
  file_path = path_of_file
  print("path of the input atsp file : ",file_path)


  # initialize variables
  n_nodes = None
  dist_matrix = []

  # open and read the file
  with open(file_path) as f:
      for line in f:
          # extract the number of nodes
          if "DIMENSION" in line:
              n_nodes = int(line.split(":")[1].strip())
          # extract the distance matrix
          elif "EDGE_WEIGHT_SECTION" in line:
              while True:
                  line = f.readline()
                  if "EOF" in line:
                      break
                  dist_matrix.append(list(map(int, line.split())))
          # continue reading until end of file
          elif "EOF" in line:
              break


  #resizing the distance matrix to a square matrix
  list_of_lists = dist_matrix

  # return list_of_lists    

  flat_list = []
  for sublist in list_of_lists:
      for item in sublist:
          flat_list.append(item)


  # return flat_list
  # Example 1D list
  lst = flat_list
  # return lst

  # Calculate the size of the square matrix
  n = len(lst)
  size = int(np.sqrt(n))

  # #print the length of the list
  # print(n)
  # Reshape the 1D list into a 2D numpy array with the desired shape
  arr = np.reshape(lst, (size, size))

  # Print the resulting array
  dist_mat = np.array(arr)

  print("No of cities : "  , n_nodes)
  print("Shape of assymentric matrix : " ,dist_mat.shape)
  return dist_mat #returns the distance matrix as a numpy array



In [5]:

def make_symmetric(distances):
  '''
  input : input a assymentric distance matrix as a numpy array
  output  : returns a symmentric distance matrix
  '''
  n = len(distances)
  distances_converted = []
  bigM = 1000000

  for cnt1 in range(0, n):
      tmp = []
      for cnt2 in range(0, n):
          tmp.append(bigM)

      for cnt2 in range(0, n):
          if cnt1 == cnt2:
              tmp.append(-bigM)    # from node to dummy node
          else:
              tmp.append(distances[cnt2][cnt1])

      distances_converted.append(tmp)

  # lower part of matrix (outgoing edges)
  for cnt1 in range(0, n):
      tmp = []
      for cnt2 in range(0, n):
          if cnt1 == cnt2:
              tmp.append(-bigM)    # from node to dummy node
          else:
              tmp.append(distances[cnt1][cnt2])

      for cnt2 in range(0, n):
          tmp.append(bigM)
      distances_converted.append(tmp)

  # check that matrix is symmetric
  for cnt1 in range(0, 2 * n):
      for cnt2 in range(0, 2 * n):
          assert (distances_converted[cnt1][cnt2] == distances_converted[cnt2][cnt1])
  
  print("Shape of symentric matrix : " ,len(distances_converted[0]) , len(distances_converted))

  return distances_converted #return a symmentric distance matrix



In [6]:


def generate_tsp_file(dist_mat, filename, num_cities):   

  '''
  input : 
  a symmentric matrix  
  directory of a .tsp file you want to generate
  number of cities in the TSP

  Output : 
  a .tsp file generated in the filename directory
  '''
  # Write the coordinates to a .tsp file
  fname = filename.split("/")[-1].split(".")[0]
  with open(filename, 'w') as f:
      f.write(f"NAME: {fname}\n")
      # f.write("COMMENT: Randomly generated TSP instance\n")
      f.write("TYPE: TSP (M.~Hofmeister)\n")
      f.write(f"DIMENSION: {num_cities*2}\n") #becase the distance matrix doubles in size when converted to a symmentric instance
      f.write("EDGE_WEIGHT_TYPE: EXPLICIT\n")
      f.write("EDGE_WEIGHT_FORMAT: FULL_MATRIX\n")
      # f.write("NODE_COORD_SECTION\n")
      f.write("DISPLAY_DATA_TYPE: NO_DISPLAY\n")
      f.write("EDGE_WEIGHT_SECTION\n")
      for i in range(len(dist_mat)):
        for j in range(len(dist_mat)):
          f.write(f'{str(dist_mat[i][j])}\t')
        f.write("\n")
      f.write("EOF\n")

In [15]:


distances = create_dist_mat_from_atsp("/content/drive/MyDrive/analytics/ft53.atsp")

new_dist = make_symmetric(distances)

generate_tsp_file(new_dist,"/content/drive/MyDrive/analytics/ft53.tsp",53)

fname = '/content/drive/MyDrive/analytics/ft53.tsp'

path of the input atsp file :  /content/drive/MyDrive/analytics/ft53.atsp
No of cities :  53
Shape of assymentric matrix :  (53, 53)
Shape of symentric matrix :  106 106


In [ ]:
from concorde.tsp import TSPSolver
from concorde.tests.data_utils import get_dataset_path
import time

start  = time.time()


solver = TSPSolver.from_tspfile(fname)

solution = solver.solve()

print(solution.found_tour)

print(solution.optimal_value +53*1000000)

print(solution.tour)
end = time.time()

In [ ]:
#automation code for checking all the instances
import os
import glob
import re
import time

base_path = os.chdir('/content/drive/MyDrive/analytics')#use os.getcwd() in windows
atsp_files =  glob.glob("/content/drive/MyDrive/analytics/*.atsp", recursive=True)

for f in atsp_files:
  
  # print("Current file : ",f)
  filename = f.split('/')[-1]
  print("File name : ",filename)
  nu = re.findall(r'\d+', filename)
  cities = [int(num) for num in nu][0]
  
  print("Num of cities : ",cities)
  
  start = time.time() 
  distances = create_dist_mat_from_atsp(f)

  new_dist = make_symmetric(distances)

  tsp_dir = f[:-5] + '.tsp'

  # print("New tsp  dir : ",tsp_dir)

  generate_tsp_file(new_dist,tsp_dir,cities)

   
  solver = TSPSolver.from_tspfile(tsp_dir)
  solution = solver.solve()
  end = time.time()

  # print("Solution tour found : ",solution.found_tour)

  print("value of solution tour : ",solution.optimal_value +cities*1000000)

  # print("Tour sequence : ",solution.tour)

  print("Total time : " , round(end - start , 3) , "secs\n")



File name :  br17.atsp
Num of cities :  17
path of the input atsp file :  /content/drive/MyDrive/analytics/br17.atsp
No of cities :  17
Shape of assymentric matrix :  (17, 17)
Shape of symentric matrix :  34 34
value of solution tour :  39.0
Total time :  0.169 secs

File name :  rbg403.atsp
Num of cities :  403
path of the input atsp file :  /content/drive/MyDrive/analytics/rbg403.atsp
No of cities :  403
Shape of assymentric matrix :  (403, 403)
Shape of symentric matrix :  806 806
